# Researching a Pairs Trading Strategy

Adapted from a Quantopian notebook by Delaney Granizo-Mackenzie

Updated for Python 3 and modern Pandas by Ken Gleason

Part of the Quantopian Lecture Series:

* [www.quantopian.com/lectures](https://www.quantopian.com/lectures)
* [github.com/quantopian/research_public](https://github.com/quantopian/research_public)

Notebook released under the Creative Commons Attribution 4.0 License.

---

Pairs trading is a nice example of a strategy based on mathematical analysis. The principle is as follows. Let's say you have a pair of securities X and Y that have some underlying economic link. An example might be two companies that manufacture the same product, or two companies in one supply chain. We'll start by constructing an artificial example.

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import seaborn as sns

import statsmodels
from statsmodels.tsa.stattools import coint
# just set the seed for the random number generator
np.random.seed(107)

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams[ 'figure.figsize' ] = ( 14, 8 )

## Explaining the Concept: We start by generating two fake securities.
We model X's daily returns by drawing from a normal distribution. Then we perform a cumulative sum to get the value of X on each day.

In [ ]:
X_returns = np.random.normal(0, 3, 100) # Generate the daily returns
# sum them and shift all the prices up into a reasonable range
X = pd.Series(np.cumsum(X_returns), name='X') + 50
X.plot(figsize = ( 16, 9 ))
plt.show()

Now we generate Y. Remember that Y is supposed to have a deep economic link to X, so the price of Y should vary pretty similarly. We model this by taking X, shifting it up and adding some random noise drawn from a normal distribution.

In [ ]:
some_noise = np.random.normal(0, 6, 100)
Y = X + 5 + some_noise
Y.name = 'Y'
pd.concat([X, Y], axis=1).plot(figsize=(16,9))
plt.show()

## Cointegration

We've constructed an example of two cointegrated series. Cointegration is a "different" form of correlation (very loosely speaking). The spread between two cointegrated timeseries will vary around a mean. The expected value of the spread over time must converge to the mean for pairs trading work work. Another way to think about this is that cointegrated timeseries might not necessarily follow a similar path to a same destination, but they both end up at this destination.

We'll plot the spread between the two now so we can see how this looks.

In [ ]:
(Y-X).plot(figsize=(16,9)) # Plot the spread
plt.axhline((Y-X).mean(), color='red', linestyle='--') # Add the mean
plt.show()

## Testing for Cointegration

That's an intuitive definition, but how do we test for this statistically? There is a convenient test that lives in `statsmodels.tsa.stattools`. We should see a very low p-value, as we've artifically created two series that are as cointegrated as physically possible.

In [ ]:
# compute the p-value of the cointegration test
# will inform us as to whether the spread btwn the 2 timeseries is stationary
# around its mean
score, pvalue, crit_vals = coint(X,Y)
print( pvalue )

### Correlation vs. Cointegration

Correlation and cointegration, while theoretically similar, are not the same. To demonstrate this, we'll show examples of series that are correlated, but not cointegrated, and vice versa. To start let's check the correlation of the series we just generated.

In [ ]:
X.corr(Y)

That's very high, as we would expect. But how would two series that are correlated but not cointegrated look? 

### Correlation Without Cointegration

A simple example is two series that just diverge.

In [ ]:
X_returns = np.random.normal(1, 1, 100)
Y_returns = np.random.normal(2, 1, 100)

X_diverging = pd.Series(np.cumsum(X_returns), name='X')
Y_diverging = pd.Series(np.cumsum(Y_returns), name='Y')

pd.concat([X_diverging, Y_diverging], axis=1).plot(figsize=(16,9))

In [ ]:
print( 'Correlation: ' + str(X_diverging.corr(Y_diverging)) )
score, pvalue, crit_val = coint(X_diverging,Y_diverging)
print( 'Cointegration test p-value: ' + str(pvalue) )
print(score)
print(crit_val)

### Cointegration Without Correlation

A simple example of this case is a normally distributed series and a square wave.

In [ ]:
Y2 = pd.Series(np.random.normal(0, 1, 1000), name='Y2') + 20
Y3 = Y2.copy()

In [ ]:
# Y2 = Y2 + 10
Y3[0:100] = 30
Y3[100:200] = 10
Y3[200:300] = 30
Y3[300:400] = 10
Y3[400:500] = 30
Y3[500:600] = 10
Y3[600:700] = 30
Y3[700:800] = 10
Y3[800:900] = 30
Y3[900:1000] = 10

In [ ]:
Y2.plot(figsize=(16,9))
Y3.plot()
plt.ylim([0, 40])

In [ ]:
# correlation is nearly zero
print( 'Correlation: ' + str(Y2.corr(Y3)) )
score, pvalue, _ = coint(Y2,Y3)
print( 'Cointegration test p-value: ' + str(pvalue) )
print( score )

Sure enough, the correlation is incredibly low, but the p-value shows perfect cointegration.

## Cointegration and Pairs
Because the securities drift towards and apart from each other, there will be times when the distance is high and times when the distance is low. The trick of pairs trading comes from maintaining a hedged position across X and Y. If both securities go down, we neither make nor lose money, and likewise if both go up. We make money on the difference of the two reverting to the mean. In order to do this we'll watch for when X and Y are far apart, then short Y and long X. Similarly we'll watch for when they're close together, and long Y and short X.

## Finding cointegrated securities
The best way to do this is to start with securities you suspect may be cointegrated and perform a statistical test. If you just run statistical tests over all pairs, you'll fall prey to multiple comparison bias.

Here's a method to look through a list of securities and test for cointegration between all pairs. It returns a cointegration test score matrix, a p-value matrix, and any pairs for which the p-value was less than 0.05.

In [ ]:
def find_cointegrated_pairs(df):
    security_names = df.columns
    n = len(security_names)
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
            S1 = df[security_names[i]]
            S2 = df[security_names[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < 0.05:
                pairs.append((security_names[i], security_names[j]))
    return score_matrix, pvalue_matrix, pairs

## Looking for Cointegrated Pairs in groups of stocks

In [ ]:
import getstock as gs
import pandas as pd
import time

In [ ]:
# skip this unless we need to run the data again
apikey = "YOURKEYHERE"
symbol_data = {}
#symbol_list = ['MSFT', 'AMZN', 'FB', 'GOOG','NFLX','SPY']
symbol_list = ['JPM', 'GS', 'MS', 'ICE', 'USB', 'PNC', 'BK', 'COF', 'INFO', 'BAC']
for symbol in symbol_list:
    symbol_data[symbol] = gs.getDailyStockPrices(symbol, apikey).adjusted_close
    time.sleep(60)

securities_df = pd.DataFrame(symbol_data)

securities_df = securities_df.loc['2015':'2019']

# save the data!
securities_df.to_pickle('bankdata.gz')

In [ ]:
# load the data
securities_df = pd.read_pickle('bankdata.gz')

In [ ]:
securities_df.tail()

Now let's see if any pairs are cointegrated.

In [ ]:
# Heatmap to show the p-values of the cointegration test between each pair of
# stocks. Only show the value in the upper-diagonal of the heatmap
# (Just showing a '1' for everything in lower diagonal)

scores, pvalues, pairs = find_cointegrated_pairs(securities_df)

sns.heatmap(pvalues, xticklabels=symbol_list, yticklabels=symbol_list, cmap='RdYlGn_r' 
                , mask = (pvalues >= 0.95))
print(pairs)

In [ ]:
# what's the best pair location?
best_pair_loc = np.unravel_index(np.argmin(pvalues, axis=None), pvalues.shape)
best_pair_name = symbol_list[best_pair_loc[0]] + ", " + symbol_list[best_pair_loc[1]]
print(str(best_pair_loc) + ": " + best_pair_name)

Looks like 'COF' and 'BAC' are cointegrated. Let's take a look at the prices to make sure there's nothing weird going on.

In [ ]:
S1 = securities_df['COF']
S2 = securities_df['BAC']

In [ ]:
score, pvalue, _ = coint(S1, S2)
pvalue

We'll plot the spread of the two series.

In [ ]:
diff_series = S1 - S2
diff_series.plot()
plt.axhline(diff_series.mean(), color='black')

The absolute spread isn't very useful in statistical terms. It is more helpful to normalize our signal by treating it as a z-score. This way we associate probabilities to the signals we see. If we see a z-score of 1, we know that approximately 84% of all spread values will be smaller.

In [ ]:
def zscore(series):
    return (series - series.mean()) / np.std(series)

In [ ]:
zscore(diff_series).plot()
plt.axhline(zscore(diff_series).mean(), color='black')
plt.axhline(1.0, color='red', linestyle='--')
plt.axhline(-1.0, color='green', linestyle='--')


### Simple Strategy: 
* Go "Long" the spread whenever the z-score is below -1.0
* Go "Short" the spread when the z-score is above 1.0
* Exit positions when the z-score approaches zero

Since we originally defined the "spread" as S1-S2:
* "Long" the spread would mean "Buy 1 dollar of S1, and Sell Short 1 dollar of S2" 
* and vice versa if you were going "Short" the spread)

This is just the tip of the iceberg, and only a very simplistic example to illustrate the concepts.  In practice you would want to compute a more optimal weighting for how many shares to hold for S1 and S2.  Some additional resources on pair trading are listed at the end of this notebook

## Moving Average
A moving average is just an average over the last $n$ datapoints for each given time. It will be undefined for the first $n$ datapoints in our series.

In [ ]:
# Get the difference in prices between the 2 stocks
difference = S1 - S2
difference.name = 'diff'

# Get the 10 day moving average of the difference
diff_mavg10 = difference.rolling(10).mean()
diff_mavg10.name = 'diff 10d mavg'

# Get the 60 day moving average
diff_mavg60 = difference.rolling(60).mean()
diff_mavg60.name = 'diff 60d mavg'

_ = pd.concat([diff_mavg60, diff_mavg10, difference], axis=1).plot()

We can use the moving averages to compute the z-score of the difference at each given time. This will tell us how extreme the difference is and whether it's a good idea to enter a position at this time. Let's take a look at the z-score now.

In [ ]:
# Take a rolling 60 day standard deviation
std_60 = difference.rolling(60).std()
std_60.name = 'std 60d'

# Compute the z score for each day
zscore_60_10 = (diff_mavg10 - diff_mavg60)/std_60
zscore_60_10.name = 'z-score'
zscore_60_10.plot()
plt.axhline(0, color='black')
plt.axhline(1.0, color='red', linestyle='--')
plt.axhline(-1.0, color='green', linestyle='--')
plt.show()

The z-score doesn't mean much out of context, let's plot it next to the prices to get an idea of what it looks like. We'll take the negative of the z-score because the differences were all negative and that's kinda confusing.

In [ ]:
two_stocks = securities_df[['COF', 'BAC']]
# Plot the prices scaled down along with the negative z-score
# just divide the stock prices by 10 to make viewing it on the plot easier
pd.concat([two_stocks/10, zscore_60_10], axis=1).plot()
plt.show()

## This notebook contained some simple introductory approaches. In practice one should use more sophisticated statistics, some of which are listed here.

* Augmented-Dickey Fuller test 
* Hurst exponent
* Half-life of mean reversion inferred from an Ornstein–Uhlenbeck process
* Kalman filters